In [ ]:
!pip uninstall -y opencv-python opencv-python-headless
!pip install opencv-contrib-python-headless==4.5.5.64

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import blob_dog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import xgboost as xgb

In [ ]:
# --- Step 1: Acquire Video Paths and Labels ---
def acquire_videos_and_labels(dataset_path):
    video_paths = []
    labels = []
    class_labels = {
        "AccidentsInaTunnel": 0,
        "FireClips": 1,
        "OtherClips": 2,
        "Smoke_Far": 3,
        "SmokeClips": 4
    }
    for folder_name, label in class_labels.items():
        folder_path = os.path.join(dataset_path, folder_name)
        for video_file in os.listdir(folder_path):
            if video_file.endswith(".mp4") or video_file.endswith(".avi"):
                video_paths.append(os.path.join(folder_path, video_file))
                labels.append(label)
    return video_paths, labels

In [ ]:
# --- Step 2: Acquire Frames from a Single Video ---
def acquire_frames_from_video(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

In [ ]:
# --- Step 3: Enhancement ---
def enhance_image(image):
    gaussian_filtered = cv2.GaussianBlur(image, (5, 5), 0)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened_image = cv2.filter2D(gaussian_filtered, -1, kernel)
    return sharpened_image

In [ ]:
# --- Step 4: Segmentation ---
def segment_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresholded

In [ ]:
# --- Step 5: Convex Hull ---
def apply_convex_hull(segmented_image):
    contours, _ = cv2.findContours(segmented_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hulls = [cv2.convexHull(contour) for contour in contours]
    hull_image = np.zeros_like(segmented_image)
    cv2.drawContours(hull_image, hulls, -1, (255), thickness=cv2.FILLED)
    return hull_image, hulls

In [ ]:
# --- Step 6: Detect Blobs ---
def detect_blobs(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blobs = blob_dog(gray, min_sigma=1, max_sigma=50, threshold=0.2)
    # Flatten blob coordinates into a single feature vector
    blob_features = np.array([len(blobs)])  # Number of blobs detected
    return blob_features

In [ ]:
# --- Step 6: Feature Extraction ---
def extract_lbp_histogram(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(257), range=(0, 256))
    hist = hist.astype("float")
    hist /= hist.sum()  # Normalize
    return hist

In [ ]:
def apply_gabor_filter(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernels = [cv2.getGaborKernel((21, 21), 5.0, theta * np.pi / 4.0, 10.0, 0.5, 0) for theta in range(4)]
    filtered_images = [cv2.filter2D(gray, cv2.CV_32F, k) for k in kernels]
    combined_gabor = np.max(filtered_images, axis=0)
    return combined_gabor

In [ ]:
def compute_optical_flow(prev_frame, next_frame):
    # Resize next_frame if dimensions don't match
    if prev_frame.shape[:2] != next_frame.shape[:2]:
        next_frame = cv2.resize(next_frame, (prev_frame.shape[1], prev_frame.shape[0]))
    
    # Convert frames to grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

    # Compute optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Calculate magnitude and angle of the flow
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Create a normalized histogram of the magnitude
    flow_hist, _ = np.histogram(magnitude.ravel(), bins=64, range=(0, 255))
    flow_hist = flow_hist.astype("float") / (flow_hist.sum() + 1e-8)  # Prevent division by zero

    return flow_hist

In [ ]:
def represent_features(frame, next_frame):
    enhanced_frame = enhance_image(frame)
    segmented_frame = segment_image(enhanced_frame)
    hull_image, hulls = apply_convex_hull(segmented_frame)
    lbp_hist = extract_lbp_histogram(enhanced_frame)
    gabor_filtered = apply_gabor_filter(enhanced_frame)
    gabor_hist, _ = np.histogram(gabor_filtered.ravel(), bins=64, range=(0, 255))
    gabor_hist = gabor_hist.astype("float") / gabor_hist.sum()
    flow_hist = compute_optical_flow(frame, next_frame)
    blob_features = detect_blobs(enhanced_frame)
    combined_features = np.hstack([lbp_hist, gabor_hist, flow_hist, blob_features])
    return combined_features

In [ ]:
def process_videos(video_paths, labels):
    all_features = []
    all_labels = []
    for video_path, label in zip(video_paths, labels):
        frames = acquire_frames_from_video(video_path)
        for i in range(len(frames) - 1):  # Process pairs of frames for optical flow
            feature_vector = represent_features(frames[i], frames[i + 1])
            all_features.append(feature_vector)
            all_labels.append(label)
    return np.array(all_features), np.array(all_labels)

In [ ]:
def train_and_evaluate(X_train, y_train, X_test, y_test):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    params = {
        "objective": "multi:softmax",
        "num_class": 5,
        "tree_method": "gpu_hist",  # Enable GPU
        "predictor": "gpu_predictor",
        "max_depth": 6,
        "learning_rate": 0.1,
        "n_estimators": 100,
    }
    
    model = xgb.train(params, dtrain, num_boost_round=100)
    y_pred = model.predict(dtest)

    print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# --- Main Script ---
dataset_path = "/kaggle/input/visifire"
video_paths, video_labels = acquire_videos_and_labels(dataset_path)

In [ ]:
# Split dataset at video level
train_videos, test_videos, train_labels, test_labels = train_test_split(
    video_paths, video_labels, test_size=0.2, random_state=42
)

In [ ]:
# Process training and test sets
X_train, y_train = process_videos(train_videos, train_labels)
X_test, y_test = process_videos(test_videos, test_labels)

In [ ]:
train_and_evaluate(X_train, y_train, X_test, y_test)